In [2]:
from ftplib import FTP_TLS
import gzip
import matplotlib.pyplot as plt
import numpy as np
import os
import csv

# FTP Config

In [4]:
host = "gdc.cddis.eosdis.nasa.gov"
port = 21  # passive mode
username = "anonymous"
password = ""

In [5]:
ftp = FTP_TLS()
ftp.connect(host, port)
ftp.login(username, password)
ftp.prot_p()
ftp.retrlines('LIST') # list files in server

lrwxrwxrwx    1 ftp      ftp             8 Sep 06  2022 GSAC -> pub/GSAC
lrwxrwxrwx    1 ftp      ftp             9 Sep 06  2022 doris -> pub/doris
lrwxrwxrwx    1 ftp      ftp            11 Sep 06  2022 glonass -> pub/glonass
lrwxrwxrwx    1 ftp      ftp             7 Sep 06  2022 gnss -> pub/gps
lrwxrwxrwx    1 ftp      ftp             7 Sep 06  2022 gps -> pub/gps
drwxrwxr-x   26 ftp      ftp          4096 Jan 01 00:16 highrate
lrwxrwxrwx    1 ftp      ftp             8 Sep 06  2022 misc -> pub/misc
lrwxrwxrwx    1 ftp      ftp            12 Sep 06  2022 products -> pub/products
drwxrwxr-x   13 ftp      ftp          4096 Feb 21  2023 pub
lrwxrwxrwx    1 ftp      ftp            11 Sep 06  2022 reports -> pub/reports
lrwxrwxrwx    1 ftp      ftp             7 Sep 06  2022 slr -> pub/slr
lrwxrwxrwx    1 ftp      ftp             8 Sep 06  2022 vlbi -> pub/vlbi


'226 Directory send OK.'

# Navigate to IGS Products

## Functions

In [8]:
def fetch_clks(week):
    try:
        base_dir = '/pub/gps/products'
        ftp.cwd(base_dir)
        
        week_folder = f"{week}"
        ftp.cwd(week_folder)
    
        # List all files in the directory
        
        files = []
        ftp.retrlines('NLST', files.append)

        clk_files = [file for file in files if file.endswith('.CLK.gz')] # CLK is case-sensitive
    
        data = []
        line_ct = 0


        for i, clk_file in enumerate(clk_files):
            local_filename = f'/tmp/{clk_file}'
            
            with open(local_filename, 'wb') as f:
                  ftp.retrbinary(f"RETR {clk_file}", f.write)

            if clk_file.endswith('.gz'):
                local_uncompressed = local_filename[:-3]
                with gzip.open(local_filename, 'rb') as gzipped_file, open(local_uncompressed, 'wb') as decompressed_file:
                     decompressed_file.write(gzipped_file.read())

            file_path = local_uncompressed 

            with open(file_path, 'r') as file:
                end_header = False
                for line in file:
                    if not end_header:
                        if 'END OF HEADER' in line:
                            end_header = True
                        continue

                    if line.startswith('AS'): 
                        columns = line.split()
                        if len(columns) >= 9:
                            clock_data_type = columns[0]
                            sv_name = columns[1]
                            epoch_year = int(columns[2])
                            epoch_month = int(columns[3])
                            epoch_day = int(columns[4])
                            epoch_hour = int(columns[5])
                            epoch_minute = int(columns[6])
                            epoch_second = float(columns[7])
                            num_data_value = int(columns[8])
                            clock_bias = float(columns[9].lower().replace('d', 'e'))
                            clock_bias_stddev = float(columns[10].lower().replace('d', 'e')) if len(columns) > 10 else np.nan
                            clock_rate = float(columns[11].lower().replace('d', 'e')) if len(columns) > 11 else np.nan
                            clock_rate_stddev = float(columns[12].lower().replace('d', 'e')) if len(columns) > 12 else np.nan
                            clock_acceleration = float(columns[13].lower().replace('d', 'e')) if len(columns) > 13 else np.nan
                            clock_acceleration_stddev = float(columns[14].lower().replace('d', 'e')) if len(columns) > 14 else np.nan
                            
                            if sv_name.startswith('G'):
                                row = {
                                    'Clock Data Type': clock_data_type,
                                    'SV Name': sv_name,
                                    'Epoch Year': epoch_year,
                                    'Epoch Month': epoch_month,
                                    'Epoch Day': epoch_day,
                                    'Epoch Hour': epoch_hour,
                                    'Epoch Minute': epoch_minute,
                                    'Epoch Second': epoch_second,
                                    'Number of Data Values to Follow': num_data_value,
                                    'Clock Bias (seconds)': clock_bias,
                                    'Clock Bias StdDev (seconds)': clock_bias_stddev,
                                    'Clock Rate (dimensionless)': clock_rate,
                                    'Clock Rate StdDev (dimensionless)': clock_rate_stddev,
                                    'Clock Acceleration (per second)': clock_acceleration,
                                    'Clock Acceleration StdDev (per second)': clock_acceleration_stddev,
                                    }
                                data.append(row)

            print(f"Done reading file {i + 1} out of {len(clk_files)}")
            
        indices = np.arange(1,len(data)+1)
        print(f"Data for GPS week {week} has been indexed.")
        
        sv_names = [entry['SV Name'] for entry in data]
        print(f"SV Names for GPS week {week} have been saved.")
        
        epoch_years = [entry['Epoch Year'] for entry in data]
        print(f"Epoch Years for GPS week {week} have been saved.")
        
        epoch_months = [entry['Epoch Month'] for entry in data]
        print(f"Epoch Months for GPS week {week} have been saved.")
        
        epoch_days = [entry['Epoch Day'] for entry in data]
        print(f"Epoch Days for GPS week {week} have been saved.")
        
        epoch_hours = [entry['Epoch Hour'] for entry in data]
        print(f"Epoch Hours for GPS week {week} have been saved.")
        
        epoch_minutes = [entry['Epoch Minute'] for entry in data]
        print(f"Epoch Minutes for GPS week {week} have been saved.")
        
        epoch_seconds = [entry['Epoch Second'] for entry in data]
        print(f"Epoch Seconds for GPS week {week} have been saved.")
        
        clock_biases = [entry['Clock Bias (seconds)'] for entry in data]
        print(f"Clock biases (seconds) for GPS week {week} have been saved.")
        
        csv_filename = f"gps_{week}.csv"
        with open(csv_filename, 'w', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                header = ['Index', 'SV Name', 'Epoch Year', 'Epoch Month', 'Epoch Day', 'Epoch Hour', 'Epoch Minute', 'Epoch Second', 'Clock Bias (seconds)']
                csvwriter.writerow(header)
                for i in range(len(data)):
                    csvwriter.writerow([indices[i], sv_names[i], epoch_years[i], epoch_months[i], epoch_days[i], epoch_hours[i], epoch_minutes[i], epoch_seconds[i], clock_biases[i]])
        print(f"CSV file for GPS week {week} has been created.")

        np.savez(f'gps_{week}.npz', satellite=sv_names, clock_bias=clock_biases)
        print(f"npz file for GPS week {week} has been created.")

        print(f"Data collection for GPS week {week} has been completed.")
            
        # Remove the downloaded files
        # os.remove(local_file)
        # os.remove(local_file[:-3])
                    
    except Exception as e:
        print(f"Error fetching data for week {week}: {e}")

## Get CLK Files

In [10]:
%%time 
fetch_clks(2309)

Done reading file 1 out of 695
Done reading file 2 out of 695
Done reading file 3 out of 695
Done reading file 4 out of 695
Done reading file 5 out of 695
Done reading file 6 out of 695
Done reading file 7 out of 695
Done reading file 8 out of 695
Done reading file 9 out of 695
Done reading file 10 out of 695
Done reading file 11 out of 695
Done reading file 12 out of 695
Done reading file 13 out of 695
Done reading file 14 out of 695
Done reading file 15 out of 695
Done reading file 16 out of 695
Done reading file 17 out of 695
Done reading file 18 out of 695
Done reading file 19 out of 695
Done reading file 20 out of 695
Done reading file 21 out of 695
Done reading file 22 out of 695
Done reading file 23 out of 695
Done reading file 24 out of 695
Done reading file 25 out of 695
Done reading file 26 out of 695
Done reading file 27 out of 695
Done reading file 28 out of 695
Done reading file 29 out of 695
Done reading file 30 out of 695
Done reading file 31 out of 695
Done reading file